In [1]:
!git clone https://github.com/mozilla/deepspeech
!mkdir training/
!python3 deepspeech/setup.py install
!pip3 install deepspeech-gpu
# !python3 deepspeech/util/taskcluster.py --target /content/deepspeech/native_client

Cloning into 'deepspeech'...
remote: Enumerating objects: 23755, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 23755 (delta 170), reused 254 (delta 151), pack-reused 23463
Receiving objects: 100% (23755/23755), 49.43 MiB | 28.47 MiB/s, done.
Resolving deltas: 100% (16304/16304), done.
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:463: UserWarning: Normalizing '0.10.0-alpha.3' to '0.10.0a3'
  warnings.warn(tmpl.format(**locals()))
running install
running bdist_egg
running egg_info
creating training/deepspeech_training.egg-info
writing training/deepspeech_training.egg-info/PKG-INFO
writing dependency_links to training/deepspeech_training.egg-info/dependency_links.txt
writing requirements to training/deepspeech_training.egg-info/requires.txt
writing top-level names to training/deepspeech_training.egg-info/top_level.txt
writing manifest file 'training/deepspeech_training.egg-info/SOURCES.txt'
reading manif

# Google cloud auth

This will install a ton of stuff on your Colab runtime to harness the power of the GPU runtime. Try running the LDC93S1 Test Script code to see the speed difference for yourself.

In [2]:
from google.colab import auth
auth.authenticate_user()

# Fetching the pre-trained model & sample audio clip

A pre-trained model is available to run and train recognition on. This section will fetch the model.

In [3]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update
!apt -qq install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   145k      0 --:--:-- --:--:-- --:--:--  145k
OK
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 10.6 MB of archives.
After this operation, 22.7 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.34.1_amd64.deb ...
Unpacking gcsfuse (0.34.1) ...
Setting up gcsfuse (0.34.1) ...


In [35]:
%%bash
BUCKET="gs://escandacc-collab/deepspeech-colab/corpus/"
if [[ $(gsutil -q stat $BUCKET) ]]
  then
    gcsfuse $BUCKET /corpus
  else
    (wget -O - https://www.caito.de/data/Training/stt_tts/es_ES.tgz | tar xfzv -) && gsutil cp -r es_ES/ gs://escandacc-collab/deepspeech-colab/corpus/
fi

Process is interrupted.


This section fetches a sample audio file to test recognition on. In this clip, the voice says "with your enquiries today. Please be informed that this"

In [ ]:
!mkdir customdata                              
!wget -O customdata/sample-1.wav https://github.com/mayukhnair/dsdatasets/raw/master/sample-1.wav
!wget -O customdata/ttssample1.wav https://github.com/mayukhnair/dsdatasets/raw/master/ttssample1.wav 

mkdir: cannot create directory ‘customdata’: File exists
--2018-06-28 12:35:33--  https://github.com/mayukhnair/dsdatasets/raw/master/sample-1.wav
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mayukhnair/dsdatasets/master/sample-1.wav [following]
--2018-06-28 12:35:33--  https://raw.githubusercontent.com/mayukhnair/dsdatasets/master/sample-1.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48700 (48K) [audio/wav]
Saving to: ‘customdata/sample-1.wav’

customdata/sample-1 100%[===================>]  47.56K  --.-KB/s    in 0.02s   

2018-06-28 12:35:33 (2.29 MB/s) - ‘customdata/

# Running recognition on the audio

This is where we import the Python bindings for DeepSpeech to perform recognition.

In [ ]:
from deepspeech.model import Model
import scipy.io.wavfile as wav

This is where the DeepSpeech model we'll run for evaluation is defined. The current values are as defined by Mozilla for their pre-trained model and will require change with other trained models.

Parameters are as specified in order:

*   Path to trained model
*   Number of MFCC (Mel-frequency cepstral coefficient) features
*   Size of context windows for producing timesteps in the input vector
*   Path to a list containing the alphabets for the language the model is trained on
*   Beam width for CTC decoder







In [ ]:
dsModel = Model("models/output_graph.pb", 26, 9, "models/alphabet.txt", 500)

This is where the model loads up the audio file and recognizes it.

In [ ]:
fs, audio = wav.read("customdata/ttssample1.wav")
output = dsModel.stt(audio,fs)
print(output)

NameError: ignored

# LDC93S1 Test Script

In [ ]:
%cd /content/deepspeech
!./bin/run-ldc93s1.sh

/content/deepspeech
+ [ ! -f DeepSpeech.py ]
+ [ ! -f data/ldc93s1/ldc93s1.csv ]
+ echo Downloading and preprocessing LDC93S1 example data, saving in ./data/ldc93s1.
+ python -u bin/import_ldc93s1.py ./data/ldc93s1
Successfully downloaded LDC93S1.wav 93638 bytes.
Successfully downloaded LDC93S1.txt 62 bytes.
+ [ -d  ]
+ python -c from xdg import BaseDirectory as xdg; print(xdg.save_data_path("deepspeech/ldc93s1"))
+ checkpoint_dir=/content/.local/share/deepspeech/ldc93s1
+ python -u DeepSpeech.py --train_files data/ldc93s1/ldc93s1.csv --dev_files data/ldc93s1/ldc93s1.csv --test_files data/ldc93s1/ldc93s1.csv --train_batch_size 1 --dev_batch_size 1 --test_batch_size 1 --n_hidden 494 --epoch 50 --checkpoint_dir /content/.local/share/deepspeech/ldc93s1
W Parameter --validation_step needs to be >0 for early stopping to work
I STARTING Optimization
I Training of Epoch 0 - loss: 378.105774
I Training of Epoch 1 - loss: 229.758408
I Training of Epoch 2 - loss: 192.815735
I Training of Epoch 3

I Training of Epoch 48 - loss: 7.441980
I Training of Epoch 49 - loss: 4.692185
I FINISHED Optimization - training time: 0:00:28
I Test of Epoch 50 - WER: 0.000000, loss: 4.03665828704834, mean edit distance: 0.000000


# Wipe the running instance clean

If you have committed errors don't seem to be fixable and want to restart from scratch, run this line. This erases everything from the Colab runtime. **Refresh this webpage after you run the line below, and ignore any prompts.**

In [ ]:
!kill -9 -1





---



# Extra code/test your stuff here

In [ ]:
!pip install pydrive

    100% |████████████████████████████████| 993kB 17.3MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /content/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
!mkdir drive
!google-drive-ocamlfuse drive

In [ ]:
!apt-get install sox
!apt-get install libsox-fmt-mp3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libgsm1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox2
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libgsm1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox2 sox
0 upgraded, 7 newly installed, 0 to remove and 4 not upgraded.
Need to get 526 kB of archives.
After this operation, 1,697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/universe amd64 libgsm1 amd64 1.0.13-4 [27.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu artful/universe amd64 libopencore-amrnb0 amd64 0.1.3-2.1 [92.0 kB]
Get:3 http://archive.ubuntu.com/ubuntu artful/universe amd64 libopencore-amrwb0 amd64 0.1.3-2.1 [45.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu artful/universe amd64 libsox2 amd64 14.4.1-5build1 [227 kB]
Get:5 http://a

Get:4 http://archive.ubuntu.com/ubuntu artful/universe amd64 libtwolame0 amd64 0.3.13-2 [46.1 kB]
Get:5 http://archive.ubuntu.com/ubuntu artful/universe amd64 libsox-fmt-mp3 amd64 14.4.1-5build1 [12.3 kB]
Fetched 288 kB in 0s (3,695 kB/s)
Selecting previously unselected package libmad0:amd64.
(Reading database ... 93496 files and directories currently installed.)
Preparing to unpack .../libmad0_0.15.1b-8ubuntu1_amd64.deb ...
Unpacking libmad0:amd64 (0.15.1b-8ubuntu1) ...
Selecting previously unselected package libid3tag0:amd64.
Preparing to unpack .../libid3tag0_0.15.1b-12_amd64.deb ...
Unpacking libid3tag0:amd64 (0.15.1b-12) ...
Selecting previously unselected package libmp3lame0:amd64.
Preparing to unpack .../libmp3lame0_3.99.5+repack1-9build1_amd64.deb ...
Unpacking libmp3lame0:amd64 (3.99.5+repack1-9build1) ...
Selecting previously unselected package libtwolame0:amd64.
Preparing to unpack .../libtwolame0_0.3.13-2_amd64.deb ...
Unpacking libtwolame0:amd64 (0.3.13-2) ...
Selecting pr

In [ ]:
!mkdir ldc
%cd /deepspeech
!ls 

mkdir: cannot create directory ‘ldc’: File exists
/deepspeech
deepspeech     libctc_decoder_with_kenlm.so  LICENSE
generate_trie  libdeepspeech.so		     native_client.tar.xz
ldc	       libdeepspeech_utils.so	     README.mozilla


In [ ]:
%cd /content
!mkdir cvd
!deepspeech/bin/import_cv.py cvd

/content
No archive "/content/cvd/cv_corpus_v1.tar.gz" - downloading...


Progress |####################################################################################################| 100.0% completed
No directory "/content/cvd/cv_corpus_v1.tar.gz" - extracting archive...


Progress |####################################################################################################| 100.0% completed

No CSV file "/content/cvd/cv-other-train.csv" - importing "/content/cvd/cv_corpus_v1/cv-other-train.csv"...
Importing mp3 files...


Progress |##########------------------------------------------------------------------------------------------| 10.5% completed

Progress |####################--------------------------------------------------------------------------------| 20.0% completed

In [ ]:
!mkdir lame
!wget -O lame/lame.tar.gz http://archive.ubuntu.com/ubuntu/pool/universe/l/lame/lame_3.99.5+repack1-3ubuntu1.debian.tar.gz

--2018-06-29 10:10:55--  http://archive.ubuntu.com/ubuntu/pool/universe/l/lame/lame_3.99.5+repack1-3ubuntu1.debian.tar.gz
Resolving archive.ubuntu.com (archive.ubuntu.com)... 91.189.88.149, 91.189.88.161, 91.189.88.162, ...
Connecting to archive.ubuntu.com (archive.ubuntu.com)|91.189.88.149|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13244 (13K) [application/x-gzip]
Saving to: ‘lame/lame.tar.gz’

lame/lame.tar.gz    100%[===================>]  12.93K  --.-KB/s    in 0s      

2018-06-29 10:10:55 (95.7 MB/s) - ‘lame/lame.tar.gz’ saved [13244/13244]



In [ ]:
!ls lame
!tar -xvzf lame/lame.tar.gz -C lame
!./lame/debian/libmp3lame0.install



debian	lame.tar.gz
debian/
debian/source/
debian/source/lintian-overrides
debian/source/format
debian/copyright
debian/lame-get-orig-source
debian/changelog
debian/lame-doc.install
debian/libmp3lame-dev.install
debian/gbp.conf
debian/lame-doc.doc-base
debian/lame.install
debian/control
debian/libmp3lame0.install
debian/rules
debian/watch
debian/patches/
debian/patches/debian-as-extra-dist.patch
debian/patches/ansi2knr2devnull.patch
debian/patches/unbreak-ftbfs-gcc4.4.patch
debian/patches/gtk1-ac-directives.patch
debian/patches/07-field-width-fix.patch
debian/patches/parallel-builds-fix.patch
debian/patches/series
debian/README.source
debian/compat
/bin/sh: 1: ./lame/debian/libmp3lame0.install: Permission denied


In [ ]:
!apt-get install lame

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  lame
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 45.7 kB of archives.
After this operation, 122 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu artful/universe amd64 lame amd64 3.99.5+repack1-9build1 [45.7 kB]
Fetched 45.7 kB in 0s (733 kB/s)
Selecting previously unselected package lame.
(Reading database ... 93526 files and directories currently installed.)
Preparing to unpack .../lame_3.99.5+repack1-9build1_amd64.deb ...
Unpacking lame (3.99.5+repack1-9build1) ...
Setting up lame (3.99.5+repack1-9build1) ...
Processing triggers for man-db (2.7.6.1-2) ...


In [ ]:
!ls

bazel.patch	   LICENSE		    tc-cpp-ds-tests.sh
bin		   models		    tc-node-tests-prod.sh
checkpoint	   native_client	    tc-node-tests.sh
data		   README.md		    tc-python-tests-prod.sh
DeepSpeech.py	   README.website.md	    tc-python-tests.sh
doc		   RELEASE.md		    tc-schedule.sh
Dockerfile	   requirements.txt	    tc-tests-utils.sh
drive		   resources		    tc-train-tests.sh
images		   taskcluster		    util
index.htm	   tc-benchmark-tests.sh    VERSION
ISSUE_TEMPLATE.md  tc-brew-tests.sh
lame		   tc-cpp-ds-tests-prod.sh


In [ ]:
!nvidia-smi

Tue Jun 26 10:25:32 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    34W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
!ls /

bin
boot
colabtools
content
datalab
dev
etc
gpu-tensorflow-1.9.0rc1-cp27-cp27mu-linux_x86_64.whl
gpu-tensorflow-1.9.0rc1-cp36-cp36m-linux_x86_64.whl
home
lib
lib64
media
mnt
opt
proc
root
run
sbin
srv
sys
tensorflow-1.9.0rc1-cp27-cp27mu-linux_x86_64.whl
tensorflow-1.9.0rc1-cp36-cp36m-linux_x86_64.whl
tf_deps
tmp
tools
usr
var


In [ ]:
%cd deepspeech
!python -u DeepSpeech.py --train_files data/ldc93s1/ldc93s1.csv --dev_files data/ldc93s1/ldc93s1.csv --test_files data/ldc93s1/ldc93s1.csv --train_batch_size 1 --dev_batch_size 1 --test_batch_size 1 --n_hidden 494 --epoch 50 --checkpoint_dir /content/.local/share/deepspeech/ldc93s1 --decoder_library_path native_client/libctc_decoder_with_kenlm.so


[Errno 2] No such file or directory: 'deepspeech'
/content/deepspeech
W Parameter --validation_step needs to be >0 for early stopping to work
I Test of Epoch 50 - WER: 0.090909, loss: 9.05056381225586, mean edit distance: 0.019231
I --------------------------------------------------------------------------------
I WER: 0.090909, loss: 9.050564, mean edit distance: 0.019231
I  - src: "she had your dark suit in greasy wash water all year"
I  - res: "she had your dark suit in greasy was water all year"
I --------------------------------------------------------------------------------


In [ ]:
!export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64:$LD_LIBRARY_PATH


In [ ]:
!wget -O cuda_9.0.176_384.81_linux.run https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda_9.0.176_384.81_linux-run
!sh cuda_9.0.176_384.81_linux.run -silent

--2018-06-26 12:34:17--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda_9.0.176_384.81_linux-run
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda_9.0.176_384.81_linux.run?eEyo-zE3gIwfvHdwafQWr18v5rsm_goLvMwTHN3vhuvJK9qBzQBGi7-fPWOQV006GDLDtjZFN1P63KBefCGOAduBLj2jVqOH5Iap5WYRQzoatQhdXgcAXKYzszd_8erbumpLocLzrSmn8FmbcnUe_f1Ng4SGxfnpgqNp5WRpYdgMrHKqKrMMIrCN [following]
--2018-06-26 12:34:18--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda_9.0.176_384.81_linux.run?eEyo-zE3gIwfvHdwafQWr18v5rsm_goLvMwTHN3vhuvJK9qBzQBGi7-fPWOQV006GDLDtjZFN1P63KBefCGOAduBLj2jVqOH5Iap5WYRQzoatQhdXgcAXKYzszd_8erbumpLocLzrSmn8FmbcnUe_f1Ng4SGxfnpgqNp5WRpYdgMrHKqKrMMIrCN
Resolving develo

In [ ]:
!apt-get install kmod
!sh cuda_9.0.176_384.81_linux.run -silent -verbose

Reading package lists... Done
Building dependency tree       
Reading state information... Done
kmod is already the newest version (24-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Installing the NVIDIA display driver...
Verifying archive integrity... OK

Uncompressing NVIDIA Accelerated Graphics Driver for Linux-x86_64 384.81........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................



ERROR: An NVIDIA kernel module 'nvidia-uvm' appears to already be loaded in

       your kernel.  This may be because it is in use (for example, by an X

In [ ]:
!cat /var/log/nvidia-installer.log

nvidia-installer log file '/var/log/nvidia-installer.log'
creation time: Tue Jun 26 12:40:11 2018
installer version: 384.81

PATH: /usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin

nvidia-installer command line:
    ./nvidia-installer
    --ui=none
    --no-questions
    --accept-license
    --disable-nouveau

Using built-in stream user interface
-> Detected 2 CPUs online; setting concurrency level to 2.
ERROR: Unable to find the module utility `insmod`; please make sure you have the package 'module-init-tools' or 'kmod' installed.  If you do have 'module-init-tools' or 'kmod' installed, then please check that `insmod` is in your PATH.


In [ ]:
!wget -O cuda9base.deb https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda9base.deb
!apt-get install dirmngr
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1704/x86_64/7fa2af80.pub
!apt-get update
!apt-get install --no-install-recommends cuda


--2018-06-27 08:22:12--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?dJ5tLyIbgUiFnPD6bdfwf_h3zD9E_aXzNd8urKJYbdEg5HYbYzQRTCAAo4ffkvrQaPUtHFzmeujx6PpANjtUtNgkuiKJ28T7CPc9FFrk7MstZUCzrtc-1QBl_GVsdUyRucgEmzOskDQU5PaSNYW70QE7aFSdhZk5af4Kn30yz0NikGKZGP9zHJvAtSN5zwbnAELnQYZd3YoWcqqfPKGQ [following]
--2018-06-27 08:22:12--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?dJ5tLyIbgUiFnPD6bdfwf_h3zD9E_aXzNd8urKJYbdEg5HYbYzQRTCAAo4ffkvrQaPUtHFzmeujx6PpANjtUtNgkuiKJ28T7CPc9FFrk7MstZU

Hit:6 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cuda is already the newest version (9.0.176-1).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.


In [ ]:
!apt install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binfmt-support clang-3.8 libclang-common-3.8-dev libclang1-3.8
  libcuinj64-8.0 libcusparse8.0 libffi-dev libgc1c2 libjsoncpp1 libllvm3.8
  libnppc8.0 libnppi8.0 libnppial8.0 libnppicc8.0 libnppicom8.0 libnppidei8.0
  libnppif8.0 libnppig8.0 libnppim8.0 libnppist8.0 libnppisu8.0 libnppitc8.0
  libnpps8.0 libnvblas8.0 libnvgraph8.0 libnvrtc8.0 libnvtoolsext1 libnvvm3
  libobjc-7-dev libobjc4 libpipeline1 libthrust-dev libtinfo-dev libvdpau-dev
  llvm-3.8 llvm-3.8-dev llvm-3.8-runtime nvidia-cuda-dev nvidia-cuda-doc
  nvidia-cuda-gdb nvidia-opencl-dev nvidia-profiler nvidia-visual-profiler
  opencl-c-headers opencl-clhpp-headers opencl-headers
Suggested packages:
  gnustep gnustep-devel clang-3.8-doc libvdpau-doc llvm-3.8-doc libcupti-dev
  nvidia-driver opencl-clhpp-headers-doc
Recommended packages:
  libnvcuvid1
The following NEW package

Get:32 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libnvblas8.0 amd64 8.0.61-1 [128 kB]
Get:33 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libnvgraph8.0 amd64 8.0.61-1 [2,336 kB]
Get:34 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libnvrtc8.0 amd64 8.0.61-1 [6,225 kB]
Get:35 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libnvtoolsext1 amd64 8.0.61-1 [32.2 kB]
Get:36 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libnvvm3 amd64 8.0.61-1 [4,236 kB]
Get:37 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 libthrust-dev all 1.8.1-1 [430 kB]
Get:38 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 nvidia-cuda-dev amd64 8.0.61-1 [243 MB]
Get:39 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 nvidia-cuda-doc all 8.0.61-1 [90.7 MB]
Get:40 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 nvidia-cuda-gdb amd64 8.0.61-1 [2,610 kB]
Get:41 http://archive.ubuntu.com/ubuntu artful/multiverse amd64 nvidia-profile

7Progress: [ 10%] [#####.....................................................] 8Selecting previously unselected package clang-3.8.
Preparing to unpack .../08-clang-3.8_1%3a3.8.1-24ubuntu7_amd64.deb ...
Unpacking clang-3.8 (1:3.8.1-24ubuntu7) ...
7Progress: [ 11%] [######....................................................] 8Selecting previously unselected package libtinfo-dev:amd64.
Preparing to unpack .../09-libtinfo-dev_6.0+20160625-1ubuntu1_amd64.deb ...
Unpacking libtinfo-dev:amd64 (6.0+20160625-1ubuntu1) ...
7Progress: [ 12%] [#######...................................................] 8Selecting previously unselected package libvdpau-dev:amd64.
Preparing to unpack .../10-libvdpau-dev_1.1.1-3ubuntu1_amd64.deb ...
7Progress: [ 13%] [#######...................................................] 8Unpacking libvdpau-dev:amd64 (1.1.1-3ubuntu1) ...
Selecting previously unselected package llvm-3.8-runtime.
Preparing to unpack .../11-llvm-3.8-runtime_1%3a3.8.1-24ubuntu7_amd64.deb ..

 [##################........................................] 8Selecting previously unselected package libnppig8.0:amd64.
Preparing to unpack .../25-libnppig8.0_8.0.61-1_amd64.deb ...
7Progress: [ 32%] [##################........................................] 8Unpacking libnppig8.0:amd64 (8.0.61-1) ...
7Progress: [ 33%] [###################.......................................] 8Selecting previously unselected package libnppim8.0:amd64.
Preparing to unpack .../26-libnppim8.0_8.0.61-1_amd64.deb ...
Unpacking libnppim8.0:amd64 (8.0.61-1) ...
7Progress: [ 34%] [###################.......................................] 8Selecting previously unselected package libnppist8.0:amd64.
Preparing to unpack .../27-libnppist8.0_8.0.61-1_amd64.deb ...
Unpacking libnppist8.0:amd64 (8.0.61-1) ...
7Progress: [ 35%] [####################......................................] 8Selecting previously unselected package libnppisu8.0:amd64.
Preparing to unpack .../28-libnppisu8.0_8.0.61-1_amd6

7Progress: [ 54%] [###############################...........................] 8Selecting previously unselected package opencl-headers.
Preparing to unpack .../43-opencl-headers_2.1-1_all.deb ...
7Progress: [ 55%] [###############################...........................] 8Unpacking opencl-headers (2.1-1) ...
Selecting previously unselected package nvidia-opencl-dev:amd64.
Preparing to unpack .../44-nvidia-opencl-dev_8.0.61-1_amd64.deb ...
7Progress: [ 56%] [################################..........................] 8Unpacking nvidia-opencl-dev:amd64 (8.0.61-1) ...
7Progress: [ 57%] [#################################.........................] 8Selecting previously unselected package nvidia-cuda-toolkit.
Preparing to unpack .../45-nvidia-cuda-toolkit_8.0.61-1_amd64.deb ...
Unpacking nvidia-cuda-toolkit (8.0.61-1) ...
7Progress: [ 58%] [#################################.........................] 8Selecting previously unselected package nvidia-visual-profiler.
Preparing to un

Setting up libobjc-7-dev:amd64 (7.2.0-8ubuntu3.2) ...
7Progress: [ 95%] [#######################################################...] 8Setting up llvm-3.8-runtime (1:3.8.1-24ubuntu7) ...
7Progress: [ 96%] [#######################################################...] 8Setting up llvm-3.8 (1:3.8.1-24ubuntu7) ...
7Progress: [ 97%] [########################################################..] 8Setting up llvm-3.8-dev (1:3.8.1-24ubuntu7) ...
Setting up clang-3.8 (1:3.8.1-24ubuntu7) ...
7Progress: [ 98%] [#########################################################.] 8Setting up nvidia-cuda-toolkit (8.0.61-1) ...
7Progress: [ 99%] [#########################################################.] 8Processing triggers for libc-bin (2.26-0ubuntu2.1) ...
Processing triggers for systemd (234-2ubuntu12.4) ...

78

In [ ]:
!wget -O cudnn7.tgz https://developer.download.nvidia.com/compute/machine-learning/cudnn/secure/v7.0.5/prod/9.0_20171129/cudnn-9.0-linux-x64-v7.tgz?dVPHcaIdVCN4DUzagcmhrWFPv1Zj-2u8tGWGnZfW4nqE85YMmJH5n5zWCeY2Ebd6fO9dM6VlRZngdcqpzlfi6INnonkvTdHb3JzXluTB1I9PhqHen7n0hIG3E_umw7kFehIQXAH7x8gzJR5eY7AflAsqz1rd3UIpXuNwWn_BJ13YahUGaQFrAdU3Ud90HW1meposV1dHhOg
!tar -xvzf cudnn7.tgz

--2018-06-27 09:50:54--  https://developer.download.nvidia.com/compute/machine-learning/cudnn/secure/v7.0.5/prod/9.0_20171129/cudnn-9.0-linux-x64-v7.tgz?dVPHcaIdVCN4DUzagcmhrWFPv1Zj-2u8tGWGnZfW4nqE85YMmJH5n5zWCeY2Ebd6fO9dM6VlRZngdcqpzlfi6INnonkvTdHb3JzXluTB1I9PhqHen7n0hIG3E_umw7kFehIQXAH7x8gzJR5eY7AflAsqz1rd3UIpXuNwWn_BJ13YahUGaQFrAdU3Ud90HW1meposV1dHhOg
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 192.229.211.70, 2606:2800:21f:3aa:dcf:37b:1ed6:1fb
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|192.229.211.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 348817823 (333M) [application/x-compressed]
Saving to: ‘cudnn7.tgz’

cudnn7.tgz          100%[===================>] 332.66M   167MB/s    in 2.0s    

2018-06-27 09:50:56 (167 MB/s) - ‘cudnn7.tgz’ saved [348817823/348817823]

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7